In [1]:
import os
%pwd

'd:\\MLOP_Project_new\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\MLOP_Project_new'

In [4]:
pip install mlflow

In [5]:
pip install dagshub

  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   ---------------------------------------  13.1/13.2 MB 74.7 MB/s eta 0:00:01
   ---------------------------------------- 13.2/13.2 MB 41.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 38.2 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# if u want to push or run  in local server
# local development: work on model locally and use MLFLOW SCRIPT to tack ur experimets.
# log data, exp, modells locally.
# conncet mlflow app and pip install it
# once done, mlflow api connect python to its app
# then using mlflow commands u can either store logs exp metric in ur local computer 
# by not specifying its tracking URI eg mlflow.set_experiment, mlflow_set_trackin uri()
# if u want to use or make these reports stores ouside not locally like we use cloud concepts, u can
# use DAGSHUB which acts AS CENTRALISED STORAGE FOR KEPPING MLFLOW LOGS REPORT AND 
# BY CONNECTING TO DAGSHUBBTO GITHUB IT HELPS IN COIDE VERSIONING AN ALL.
# AFTER THAT U CAN USE GITHUBACTION TO CI CD PURPOSE

#SINCE HERE WE USE CENTRALLISED STORAGE FOR MLFLOW REPORTS AN LOGS IE DAGSHUB, 
# we will first install dagshub to conncet locally and then authorise it, then connect an dcreate 
# new repo in dagshub by connecting to github repo. 


# then go to remote, take its code to SET UP DAGSHUB UISNG:set up dagshub
# "COPYING CODE: DAGSHUB.init("repo owner....")".git.
#THIS WILL PUSH IT TO CENTRALLISED STORAGE


#ALSO U NEED TO ADD ITS URI TACKING : GO TO DAGHUB WITH GITHUB, AND COPY THE URI 
# USING MLFLOW_SET_TRACKING_URI(HTTPS....). THIS WILL PUBLISH ALL REPORTS METRICS LOGS TO THIS URL IE CETRAL

# U COULD SEE ERORRIN PRMIION:
# ADD ENVIROMENT USRNAME PASSWORD, PUBLIC KEY ID.. TRACKING URI: EG 
# IMPORT OS
# os.environ["tracking usrname"] = "learn.."
#         ["password"] = ""


In [ ]:
# set new uri
# log in to dagshub(deploying not locally)

import dagshub
# dagshub owner 
# running 
dagshub.init(repo_owner='superseupper2014', repo_name='mlflow', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [6]:
os.environ["MLFLOW_TRACKING_URI"] = 'https://dagshub.com/superseupper2014/mlflow.mlflow'
os.environ["MLFLOW_TRACKING_USERNAME"] = 'superseupper2014'
os.environ["MLFLOW_TRACKING_PASSWORD"] = '6e569e2ab628afe7e28a8e56dab9f965978d59eb'


In [29]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [30]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri='https://dagshub.com/superseupper2014/mlflow.mlflow',
           
        )

        return model_evaluation_config


In [32]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [33]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [28]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-08 11:48:13,577: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-08 11:48:13,583: INFO: common: yaml file: params.yaml loaded successfully]


[2024-12-08 11:48:13,591: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-08 11:48:13,596: INFO: common: created directory at: artifacts]
[2024-12-08 11:48:13,599: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-08 11:48:15,820: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


d:\MLOP_Project_new\venv\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'ElasticnetModel'.
2024/12/08 11:48:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


In [34]:
# version 2 experimental changing params
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-08 12:55:33,891: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-08 12:55:33,935: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-08 12:55:33,987: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-08 12:55:33,993: INFO: common: created directory at: artifacts]
[2024-12-08 12:55:34,001: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-08 12:55:34,430: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /superseupper2014/mlflow.mlflow/api/2.0/mlflow/runs/create]
[2024-12-08 12:55:34,587: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/12/08 12:56:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
